# Wave propagation (hyperbolic PDE)

## libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from matplotlib import animation
from IPython.display import HTML

## initialization

In [74]:
COURANT = 0.7
X_max = 1
X_min = - X_max
T_min = 0

NUMBER_X = 100
NUMBER_T = 200

T_max =  COURANT * (X_max - X_min) * NUMBER_T / NUMBER_X + T_min

In [115]:
def create_arrays(number_x, number_t):
    X_RANGE = np.linspace(X_min, X_max, number_x + 1)
    T_RANGE = np.linspace(T_min, T_max, number_t + 1)

    X_STEP = X_RANGE[1] - X_RANGE[0]
    T_STEP = T_RANGE[1] - T_RANGE[0]
    X, T = np.meshgrid(X_RANGE, T_RANGE, indexing='ij')
    return X, T

    
def animation_plot(E):
    T_STEP = T[0, 1] - T[0, 0]
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.axis([X_min, X_max, E.min(), E.max()])

    l1, = ax.plot(X[:,0], E[:, NUMBER_T - 1], lw=3, color='black', label=NUMBER_T)
    l2, = ax.plot(X[:,0], E_exact[:, NUMBER_T - 1], lw=2, color='red')
    ax.grid(c='#aaaaaa', ls='--')

    ax.set_ylabel("f(x)")
    ax.set_xlabel("x")

    def animate(i):
        l1.set_data(X[:,0], E[:, i])
        l1.set_label('t = ' + '%.3f' % (i * T_STEP))
        l2.set_data(X[:,0], E_exact[:, i])

        legend = plt.legend(loc='upper right')
        return l1, l2, legend

    ani = animation.FuncAnimation(fig, animate, frames= np.arange(0, NUMBER_T, 5), interval=60)

    #ani.save("solution.mp4")
    return HTML(ani.to_jshtml())


## main procedure

In [116]:
X, T = create_arrays(NUMBER_X, NUMBER_T)
